In [152]:
import pandas as pd

In [153]:
"""
Weight free throw effect based on how many shots taken
"""
def ft_stat(player_percentage, player_attempts):
    league_percentage = .798
    return (player_percentage - league_percentage) * player_attempts

"""
Weight field goal effect based on how many shots taken
"""
def fg_stat(player_percentage, player_attempts):
    league_percentage = .47
    return (player_percentage - league_percentage) * player_attempts

"""
Turn players into list of dicts
"""
def create_player_list(players):
    fg, ft = 0, 0
    players_stats = {} #stats for a single player
    new_players = [] #list of dicts that holds player and weighted ft and fg
    for i in range(len(players)):
        if i % 2 == 0:
            player_stats = players[i][1].copy(deep=True).to_dict()
        else:
            ft = players[i][1]["FT%"]
            fg = players[i][1]["FG%"]
            ft = ft.split("/")
            fg = fg.split("/")
            ft_attempts = ft[1][:-1]
            fg_attempts = fg[1][:-1]
            player_stats["weighted_ft"] = ft_stat(float(player_stats["FT%"]), float(ft_attempts))
            player_stats["weighted_fg"] = fg_stat(float(player_stats["FG%"]), float(fg_attempts))
            new_players.append(player_stats)
            player_stats = {}
    return new_players
"""
Get a list of all players that were drafted. TODO: Change to lg instead of lg1 during the draft
@return (list of str) - player names that were drafted
"""
def drafted():
    list = []
    for player in lg.taken_players():
        list.append(player['name'])
    return list
"""
Remove all drafted players from the dataframe
pd (pandas dataframe) - dataframe of all available players
drafted_players (List of str) - players that were drafted
"""
def remove_drafted_players(df, drafted_players):
    return df[~df.PLAYER.isin(drafted_players)]

#Turnovers should be negative to make it subtract from total_val
required_stats = {"FG%": .47, "FT%": .798, "3PM": 1.17, "PTS": 15.137, "TREB": 6.241, "AST": 3.748, "STL": 1.132, "BLK": .862, "TO": -1.7}

"""
Albert's personal rankings that takes into considerations punts
Ranking Methodology
Number of Games played * by the following
1. Value per stat
2. Remove all categories I punt
@param pd (pandas dataframe) - dataframe of all available players
@param punts(list of str) - stats to punt

@return pandas dataframe with additional column: Albert's ranking
"""
def alberts_ranking(df, punts):
    df = df.to_dict('records')
    for row in df:
        total_val = 0
        for stat in row.items():
            if stat[0] in required_stats and stat[0] not in punts:
                total_val += min(stat[1] / required_stats[stat[0]], 3)
        row['total_val'] = total_val * row['GP']
    return pd.DataFrame(df)


In [154]:
from yahoo_oauth import OAuth2
import json
creds = {'consumer_key': "dj0yJmk9U25NRHowdE16MFNHJmQ9WVdrOVNuaGtSa3RaYTBVbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTFh", 'consumer_secret': 'd83900e9c38441740e701ea87bf4b437f07a501c'}
with open("sample.json", "w") as outfile:  
    json.dump(creds, outfile) 
oauth = OAuth2(None, None, from_file='sample.json')

import yahoo_fantasy_api
lg = yahoo_fantasy_api.game.Game(oauth, "nba").to_league("402.l.5116")

lg1 = yahoo_fantasy_api.game.Game(oauth, "nba").to_league("402.l.441")

[2020-12-21 21:21:37,007 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-12-21 21:21:37,011 DEBUG] [yahoo_oauth.oauth.handler] AUTHORIZATION URL : https://api.login.yahoo.com/oauth2/request_auth?redirect_uri=oob&response_type=code&client_id=dj0yJmk9U25NRHowdE16MFNHJmQ9WVdrOVNuaGtSa3RaYTBVbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTFh


Enter verifier : xwuhycr


In [155]:
players = pd.read_excel('hashtag.xlsx', index_col=0)

alberts_players = players.copy(deep=True)
players = [i for i in alberts_players.iterrows()]
#Add weighted free throw and fg stats to table

players = create_player_list(players)
df = pd.DataFrame(players).sort_values("TOTAL", ascending=False)

In [156]:
ranking = alberts_ranking(remove_drafted_players(df, drafted()), ["FG%", "TO"])
ranking.sort_values("total_val", ascending=False)

,3PM,AST,BLK,FG%,FT%,GP,MPG,PLAYER,PTS,R#,STL,TO,TOTAL,TREB,total_val,weighted_fg,weighted_ft
25,1.98,2.09,0.39,0.411,0.788,70.0,28.7,Dillon Brooks,16.09,181.0,0.90,1.69,-1.71,3.27,425.026638,-0.85491,-0.02780
1,1.66,2.44,0.52,0.483,0.813,70.0,24.2,Nemanja Bjelica,10.22,121.0,0.70,1.22,0.24,5.50,410.667106,0.10335,0.01605
20,1.91,1.31,0.55,0.414,0.714,70.0,24.9,Danny Green,8.23,173.0,1.20,0.90,-1.50,3.41,396.546122,-0.39368,-0.05880
19,2.54,2.40,0.13,0.427,0.816,67.0,26.8,Svi Mykhailiuk,11.54,172.0,0.89,1.39,-1.44,2.40,396.492153,-0.39173,0.02736
3,1.76,2.71,0.41,0.454,0.754,64.0,27.9,Kevin Porter Jr.,13.85,136.0,1.06,2.43,-0.18,4.30,396.044190,-0.17696,-0.11968
15,1.55,1.61,0.62,0.459,0.736,70.0,26.1,Dorian Finney-Smith,9.04,165.0,0.77,0.96,-0.99,5.34,387.027782,-0.07711,-0.08928
9,1.27,2.44,0.49,0.455,0.756,70.0,28.2,Royce O'Neale,6.68,155.0,1.02,0.99,-0.80,5.37,381.856981,-0.07950,-0.03276
2,1.67,3.76,0.52,0.432,0.670,60.0,26.9,Patrick Beverley,8.37,124.0,1.15,1.36,0.19,5.44,378.834322,-0.26638,-0.12032
0,1.69,3.07,0.29,0.412,0.880,60.0,28.7,Alec Burks,13.56,113.0,0.98,1.58,0.44,4.65,372.560835,-0.64090,0.25994
26,1.20,3.61,0.44,0.406,0.818,68.0,24.6,Frank Ntilikina,7.81,183.0,1.11,1.54,-1.78,2.52,368.874567,-0.43520,0.02640
